#Installing The Libraries


In [1]:
#downloading geopandas library
!pip install geopandas
!pip install folium
!pip install altair
!pip install matplotlib==3.4.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 10.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/co

#**Importing The Libraries**

In [124]:
import pandas as pd
import requests
import geopandas
import numpy as np
import statsmodels.api as sm
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, ttest_ind
from scipy.stats import mannwhitneyu
import seaborn as sns
import folium
import plotly.express as px
import altair as alt


# **Extrating Data Through API**



in the cell below there is a class that you can pass in the main url and the api key once you made the object you can use the method and give it the extension url and the params then you will receive data

In [451]:


class main:
  
  def __init__(self, api, endpoint):
    self.headers = {"Authorization": "Bearer " + api}

    self.endpoint = endpoint


class apis(main):

  def get(self, url, **params):
    """
    **params** 
    year 
    classification  (HighSchool, JUCO, PrepSchool)
    position Position abbreviation filter
    state State or province abbreviation filter
    team Committed team filter (required if year not specified)
    """
    
    #add at least (year,team) in the params)
    
    return requests.get(self.endpoint + url , params=params, headers=self.headers).json()


In [452]:
BEGIN_YEAR = 2011
END_YEAR = 2023

**Requesting the recruiting players dataset through the api object**
setting the param key year to a range from 2011 to 2022 included
reading the data by json_normalize it makes one of the columns not having a json objects it will split them up to more than one column in the dataframe and I did Concat each dataframe to get all the years required

In [453]:
# making an api object
api = ""
ob = apis(api, "https://api.collegefootballdata.com/")

# getting recruited players for the year 2022
# df = pd.DataFrame(ob.get("recruiting/players",year=2022))
df = pd.concat([pd.json_normalize(ob.get("recruiting/players", year=year)) for year in range(BEGIN_YEAR, END_YEAR + 1)])


# Saving the primary dataset locally
and showing a sample of it

In [454]:
#saving the primary dataset 
df.to_csv("nfl_data.csv")

In [455]:
df.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,weight,stars,rating,city,stateProvince,country,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,255.0,5,1.0000,Rock Hill,SC,USA,34.9248667,-81.0250784,45091
1,19572,-1000934,HighSchool,2011,2.0,Cyrus Kouandjio,DeMatha Catholic,Alabama,OT,79.0,315.0,5,0.9993,Hyattsville,MD,USA,38.9529442,-76.9408647,24033
2,19573,-1016566,HighSchool,2011,3.0,La'el Collins,Redemptorist,LSU,OT,77.0,306.0,5,0.9976,Baton Rouge,LA,USA,30.4459596,-91.18738,22033
3,19574,517390,HighSchool,2011,4.0,Karlos Williams,Ridge Community,Florida State,S,74.0,210.0,5,0.9976,Davenport,FL,USA,28.1614046,-81.6017417,12105
4,19575,517222,HighSchool,2011,5.0,Anthony Johnson,O Perry Walker,LSU,DT,75.0,294.0,5,0.9973,New Orleans,LA,USA,29.9499323,-90.0701156,22071


In [456]:

df.dropna(subset=['committedTo','school','name'],inplace=True)

# Downloading the secondary dataset 
one dataset can be downloaded from the cell below the other one needs to be downloaded through the link and move it to the project directory 

In [457]:
#downloading the files income data based on city and cities based on coordinates

"""
you will have to download the cities based on coordinates data
here is the link to it and move the file to the project directory

https://www.unitedstateszipcodes.org/zip_code_database.csv?download_auth=a5443d423dfb4db4fe077643f04b2f8c

"""
!wget https://www.irs.gov/pub/irs-soi/19zpallagi.csv

--2023-02-21 15:21:37--  https://www.irs.gov/pub/irs-soi/19zpallagi.csv
Resolving www.irs.gov (www.irs.gov)... 104.102.134.224, 2600:1408:c400:c8b::f50, 2600:1408:c400:c90::f50, ...
Connecting to www.irs.gov (www.irs.gov)|104.102.134.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘19zpallagi.csv.4’

19zpallagi.csv.4        [         <=>        ] 190.40M   108MB/s    in 1.8s    

2023-02-21 15:21:39 (108 MB/s) - ‘19zpallagi.csv.4’ saved [199650533]



The 19zpallagi dataset has only the income based on zip codes and for the zip_code_database dataset has zip codes, cities, coordinates, etc.


In [458]:
# IRS income dataset
income = pd.read_csv("19zpallagi.csv")

# Coordinates with zip codes
zips_coordinates = pd.read_csv("zip_code_database.csv")

In [459]:
# isolating and removing non-US recruits

non_us_players = [df['hometownInfo.latitude'].isna()]
df = df[df['hometownInfo.latitude'].isna() == False]

In [460]:
zips_coordinates.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [461]:
income.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,1,778210.0,491030.0,84770.0,189600.0,712890.0,30670.0,...,0.0,0.0,62720.0,51936.0,671860.0,1700965.0,669570.0,1694792.0,1980.0,3512.0
1,1,AL,0,2,525940.0,247140.0,123910.0,139860.0,481760.0,18960.0,...,0.0,0.0,85860.0,122569.0,438020.0,1274802.0,435210.0,1266557.0,3670.0,7410.0
2,1,AL,0,3,285700.0,105140.0,128140.0,44560.0,260570.0,10670.0,...,0.0,0.0,73980.0,154932.0,212040.0,575315.0,208470.0,564202.0,5020.0,13653.0
3,1,AL,0,4,179070.0,38820.0,123110.0,13740.0,164300.0,5020.0,...,0.0,0.0,51330.0,139065.0,126850.0,401581.0,123310.0,388749.0,3040.0,10377.0
4,1,AL,0,5,257010.0,28180.0,216740.0,7150.0,236850.0,8400.0,...,90.0,141.0,104290.0,460071.0,152790.0,598248.0,144640.0,539385.0,9180.0,56257.0


Here I did merge two datasets the income one and zip_coordinates by zipcode then did groupby by zipcode and applied mean to the average income per household to get the mean income per household in a city 

In [462]:
# avg income by zipcode

median_income_by_zipcode = income.merge(zips_coordinates, left_on=['zipcode'], right_on=['zip']).groupby("zipcode").A02650.mean().reset_index()


In [463]:
# merging with other info

merged_income_zipcor = zips_coordinates.merge(median_income_by_zipcode, left_on=['zip'], right_on=['zipcode'])

In [464]:
# adjusting coordinates for merging
merged_income_zipcor['adj_lat'] = merged_income_zipcor['latitude'].apply(lambda x: float(str(x).split(".")[0]))

merged_income_zipcor['adj_lng'] = merged_income_zipcor['longitude'].apply(lambda x: float(str(x).split(".")[0]))

In [465]:
# getting average income by city
income_by_city_state_coord = merged_income_zipcor.groupby(['primary_city', 'state', 'adj_lat', 'adj_lng']).A02650.mean().reset_index()

income_by_city_state_coord['A02650'] = income_by_city_state_coord['A02650'].round()

income_by_city_state_coord = income_by_city_state_coord.rename(columns={"A02650" : "Average Income"})


Here I Changed the type of the two columns latitude and longitude. 
to float and did rounded it down 

In [466]:
df['lat_adj'] = df['hometownInfo.latitude'].apply(lambda x: float(str(x).split(".")[0]))

df['lng_adj'] = df['hometownInfo.longitude'].apply(lambda x: float(str(x).split(".")[0]))


In [467]:
df.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,...,stars,rating,city,stateProvince,country,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode,lat_adj,lng_adj
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,...,5,1.0000,Rock Hill,SC,USA,34.9248667,-81.0250784,45091,34.0,-81.0
1,19572,-1000934,HighSchool,2011,2.0,Cyrus Kouandjio,DeMatha Catholic,Alabama,OT,79.0,...,5,0.9993,Hyattsville,MD,USA,38.9529442,-76.9408647,24033,38.0,-76.0
2,19573,-1016566,HighSchool,2011,3.0,La'el Collins,Redemptorist,LSU,OT,77.0,...,5,0.9976,Baton Rouge,LA,USA,30.4459596,-91.18738,22033,30.0,-91.0
3,19574,517390,HighSchool,2011,4.0,Karlos Williams,Ridge Community,Florida State,S,74.0,...,5,0.9976,Davenport,FL,USA,28.1614046,-81.6017417,12105,28.0,-81.0
4,19575,517222,HighSchool,2011,5.0,Anthony Johnson,O Perry Walker,LSU,DT,75.0,...,5,0.9973,New Orleans,LA,USA,29.9499323,-90.0701156,22071,29.0,-90.0


In [468]:
# joining together for final df
print(len(df))

final = df.merge(income_by_city_state_coord, left_on=['city', 'stateProvince', 'lat_adj', 'lng_adj'],
                 right_on=['primary_city', 'state', 'adj_lat', 'adj_lng'])

36279


In [469]:
final.head()

,id,athleteId,recruitType,year,ranking,name,school,committedTo,position,height,...,hometownInfo.latitude,hometownInfo.longitude,hometownInfo.fipsCode,lat_adj,lng_adj,primary_city,state,adj_lat,adj_lng,Average Income
0,19571,-1032080,HighSchool,2011,1.0,Jadeveon Clowney,South Pointe,South Carolina,WDE,77.0,...,34.9248667,-81.0250784,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0
1,20160,512424,HighSchool,2011,591.0,Justin Worley,Northwestern,Tennessee,PRO,76.0,...,34.9248667,-81.0250784,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0
2,20244,-1007350,HighSchool,2011,676.0,Roderick Byers,Northwestern,Clemson,SDE,75.0,...,34.9248667,-81.0250784,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0
3,20875,-1032025,HighSchool,2011,1345.0,Gerald Dixon Jr.,Northwestern,South Carolina,DT,74.0,...,34.9248667,-81.0250784,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0
4,21202,-1032025,HighSchool,2011,1672.0,Gerald Dixon,South Pointe,South Carolina,SDE,74.0,...,34.9248667,-81.0250784,45091,34.0,-81.0,Rock Hill,SC,34.0,-81.0,269076.0


In [470]:
# data for win%
cfb_results = pd.concat([pd.json_normalize(ob.get("records", year=year)) for year in range(BEGIN_YEAR,END_YEAR+1) ])


In [471]:
# data for productivity metrics
cfb_player_ppa = pd.concat([pd.json_normalize(ob.get("ppa/players/season" ,year=year)) for year in range(BEGIN_YEAR,END_YEAR+1)])

In [472]:
position_dict = {
    'RET' : "returner",
    "FB" : "full back",
    "LS" : "long snapper",
    "EDGE" : "edge",
    "QB" : "quarterback",
    "IOL" : "interior offensive line",
    "P" : "punter",
    "LB" : "linebacker",
    "DL" : "defensive lineman",
    "K" : "kicker",
    "APB" : "all purpose back",
    "OC" : "center",
    "DUAL" : "quarterback",
    "PRO" : "quarterback",
    "WDE" : "weak side defensive end",
    "ILB" : "inside linebacker",
    "SDE" : "strong side defensive end", 
    "TE" : "tight end",
    "OG" : "offensive guard",
    "DT" : "defensive tackle",
    "OLB" : "outside linebacker",
    "RB" : "running back",
    "S" : "safety",
    "ATH" : "athlete",
    "OT" : "offensive tackle",
    "CB" : "cornerback",
    "WR" : "wide receiver"
}

In [473]:
special_teams = ['RET', 'LS', 'P', 'K']
defensive_line = ['EDGE', 'DL', 'WDE', 'SDE', 'DT']
linebackers = ["LB", "ILB", "OLB"]
secondary = ["CB", "S"]
backs = ['FB', "APB", "RB"]
receivers = ["WR", "TE"]
offensive_line = ["IOL", "OC", "OG", "OT"]
other = ["ATH"]
qb = ['QB', 'DUAL', 'PRO']

In [474]:
def assign_position_group(position):
    if position in special_teams:
        return "special teams"
    
    elif position in defensive_line:
        return "defensive line"
    
    elif position in linebackers:
        return "linebackers"
    
    elif position in secondary:
        return "secondary"
    
    elif position in backs:
        return "backs"
    
    elif position in receivers:
        return "receiver"
    
    elif position in offensive_line:
        return "offensive line"
    
    elif position in qb:
        return "quarterback"
    
    else:
        return "other"

In [475]:
cfb = final.copy()

In [476]:
cfb['position'] = cfb['position'].apply(lambda x: assign_position_group(x))

In [477]:
# removing schools with little representation

cfb = cfb[cfb['committedTo'].isin(cfb.groupby("committedTo").size().sort_values().head(100).index) == False]

In [478]:
# slimming down dfs for visual ease

relevant_columns_income = ['STATE', 'zipcode', 'A02650']

relevant_columns_linking_info = ['state', 'zip', 'primary_city', 'latitude', 'longitude']

income = income[relevant_columns_income]

zips_coordinates = zips_coordinates[relevant_columns_linking_info]

In [479]:
# removing unranked
cfb = cfb[cfb['ranking'].isna() == False]

In [480]:
# removing those with missing info
cfb = cfb[(cfb['height'].isna() == False) & (cfb['weight'].isna() == False)]

In [481]:
# cfb dupe check
cfb = cfb[cfb[['name', 'hometownInfo.longitude']].duplicated() == False]

In [482]:
# new column for overall ranking across all years for players
cfb['entire_dataset_ranking'] = cfb['rating'].rank(ascending=False)

<ipython-input-482-a912ebb25644>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Linking with team results

In [483]:
cfb_results['win%'] = cfb_results['total.wins'] / cfb_results['total.games']

In [484]:
final_with_results = cfb.merge(cfb_results[['team', 'year', 'conference', 'division', 'win%']],
                               left_on=['committedTo', 'year'], right_on=['team', 'year'], how='left')

In [485]:
final_with_results = final_with_results[final_with_results['win%'].isna() == False]

In [486]:
# agg correlation results
final_with_results.groupby("team")[['win%', "rating", 'stars']].mean().corr()

,win%,rating,stars
win%,1.000000,0.586205,0.586827
rating,0.586205,1.000000,0.996659
stars,0.586827,0.996659,1.000000


In [487]:
print(final_with_results.columns)

Index(['id', 'athleteId', 'recruitType', 'year', 'ranking', 'name', 'school',
       'committedTo', 'position', 'height', 'weight', 'stars', 'rating',
       'city', 'stateProvince', 'country', 'hometownInfo.latitude',
       'hometownInfo.longitude', 'hometownInfo.fipsCode', 'lat_adj', 'lng_adj',
       'primary_city', 'state', 'adj_lat', 'adj_lng', 'Average Income',
       'entire_dataset_ranking', 'team', 'conference', 'division', 'win%'],
      dtype='object')


In [488]:
# getting correlations by position
outcome_corr_by_position = final_with_results[['position', 'win%', 'rating', 'Average Income', 'stars']].groupby("position").corr().reset_index()


In [489]:
qbs_and_dline = outcome_corr_by_position[outcome_corr_by_position['position'].isin(['quarterback', 'defensive line'])]

In [490]:
# correlation for positions

outcome_corr_by_position[outcome_corr_by_position['level_1'] == 'stars'].sort_values(by='win%').rename(columns={"level_1" : "variable"})[['position', 'variable', 'win%']]


,position,variable,win%
35,special teams,stars,0.178222
23,quarterback,stars,0.216964
19,other,stars,0.224832
27,receiver,stars,0.251369
31,secondary,stars,0.260664
11,linebackers,stars,0.267344
15,offensive line,stars,0.275165
3,backs,stars,0.279936
7,defensive line,stars,0.284253


In [491]:
qbs_and_dline.set_index(["position", "level_1"])

win%    rating  Average Income     stars
position       level_1                                                     
defensive line win%            1.000000  0.320806       -0.025814  0.284253
               rating          0.320806  1.000000        0.002947  0.913763
               Average Income -0.025814  0.002947        1.000000  0.007177
               stars           0.284253  0.913763        0.007177  1.000000
quarterback    win%            1.000000  0.240682        0.015285  0.216964
               rating          0.240682  1.000000        0.099299  0.900656
               Average Income  0.015285  0.099299        1.000000  0.083699
               stars           0.216964  0.900656        0.083699  1.000000

# Linking with player productivity metrics

In [492]:
# making sure players have played more than 50 snaps
cfb_player_ppa = cfb_player_ppa[cfb_player_ppa['countablePlays'] > 50]

In [493]:
# filling na with 0
cfb_player_ppa.fillna(0.0, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5176: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [494]:
# getting average productivity by player
productivity_by_player = cfb_player_ppa.groupby(["name", "team", "position"]).mean().reset_index()

In [495]:
# linking with recruiting info
ppa_and_recruiting = final[['year', 'ranking', 'height', 'weight',
                              'stars', 'rating', 'name', 'committedTo']].merge(productivity_by_player, left_on=['name', 'committedTo'], right_on=['name', 'team'])

In [496]:
# focuing on QBs
qbs_ppa = ppa_and_recruiting[ppa_and_recruiting['position'] == 'QB'].drop(['year', 'ranking', 'height', 'weight', 'season'], axis=1)

In [497]:
# creating correlation matrix
qb_ppa_corr = qbs_ppa.corr()[['stars']].reset_index().rename(columns={'index' : "variable"})

In [498]:
qb_ppa_corr

,variable,stars
0,stars,1.000000
1,rating,0.911361
2,countablePlays,0.125481
3,averagePPA.all,0.176399
4,averagePPA.pass,0.156385
5,averagePPA.rush,0.053186
6,averagePPA.firstDown,0.175686
7,averagePPA.secondDown,0.142901
8,averagePPA.thirdDown,0.078258
9,averagePPA.standardDowns,0.167317


# Adding a column a color column and geometry column

the first column color giving green color for the high income level, blue for the middle income level and red for the low income level

the second column will be geometry column it will be for ploting with follium it has the latitude nad longitude. 

In [500]:
top_10 = final['Average Income'].quantile(0.9)
print("Top 10%", top_10)

top_25 = final['Average Income'].quantile(0.75)
print("Top 25%", top_25)

top_50 = final['Average Income'].quantile(0.5)
print("Top 50%", top_50)

top_75 = final['Average Income'].quantile(0.25)
print("Top 75%", final['Average Income'].quantile(0.25))

top_90 = final['Average Income'].quantile(0.1)

Top 10% 391258.0
Top 25% 257687.0
Top 50% 169904.0
Top 75% 102868.0


In [ ]:
col = 'Average Income'
conditions = [final[col] >= top_10,
              (final[col] < top_10) & (final[col] >= top_25),
              (final[col] < top_25) & (final[col] >= top_50),
              (final[col] < top_50) & (final[col] >= top_75),
              final[col] < top_75]

choices = ['top 10 pct', "top quartile", "second quartile", 'third quartile', 'fourth quartile']
final['Quartile Level'] = np.select(conditions, choices, default=np.nan)
final.head()

# On average, how many prospects fall into each star rating category?

In [ ]:
# Calculate Total and Average prospects per year by star ranking

# Get total counts for each star level
yearly = final.groupby(['year', 'stars'])['id'].count()

# Convert into df
yearly_df = pd.DataFrame(yearly)

# Review yearly totals
yearly_df.tail(15)

# Calculate average number of prospects per year for each star ranking
yr_avg = pd.DataFrame(yearly_df.groupby(['stars'])['id'].mean().round(0))

# Calculate average total number of prospects per year
print('Average total prospects per year',yr_avg.sum())

# Show averages by star ranking
yr_avg

In [ ]:
# Create position group mapping

position_map = {'QB':'Quarterback', 
                'DUAL':'Quarterback',
                'PRO':'Quarterback',
                'RB':'Running Back',
                'FB':'Running Back',
                'APB':'Running Back',
                'WR':'Wide Receiver',
                'TE':'Wide Receiver',
                'IOL':'Offensive Line',
                'OC':'Offensive Line',
                'OG':'Offensive Line',
                'OT':'Offensive Line',
                'EDGE':'Def Line',
                'DL':'Def Line',
                'WDE':'Def Line',
                'SDE':'Def Line',
                'DT':"Def Line",
                'LB':'Linebacker',
                'ILB':'Linebacker',
                'OLB':'Linebacker',
                'S':'Cornerback',
                'CB':'Cornerback'
               }

In [ ]:
# Map position groups dictionary to 'position' column and create new 'mapped position' column

# Create copy of df
df_mapped_positions = final.copy()

# Map position group names to position column
df_mapped_positions['mapped_position'] = df_mapped_positions['position'].map(position_map)
#df_mapped_positions.head()

# 5-star recruits ------------------------------------------

# Get subset of data for only 5-star recruits
stars_5 = df_mapped_positions[df_mapped_positions['stars'] == 5]
print("Total 5 star Recruits (all positions): ", len(stars_5))

# Get counts by position
position_5 = stars_5.groupby('mapped_position')['mapped_position'].count()
#position_5

# 4-star recruits ------------------------------------------

# Get subset of data for only 4-star recruits
stars_4 = df_mapped_positions[df_mapped_positions['stars'] == 4]
print("Total 4 star Recruits (all positions): ", len(stars_4))

# Get counts by position
position_4 = stars_4.groupby('mapped_position')['mapped_position'].agg('count')
#position_4

# 3-star recruits ------------------------------------------

# Get subset of data for only 3-star recruits
stars_3 = df_mapped_positions[df_mapped_positions['stars'] == 3]
print("Total 3 star Recruits (all positions): ",len(stars_3))

# Get counts by position
position_3 = stars_3.groupby('mapped_position')['mapped_position'].count()
#position_3

# Combine 5, 4 and 3 star data ------------------------------

all_ranking = pd.DataFrame([position_5,
                            position_4,
                            position_3])

# Transpose to get star rankings as columns
all_rankings_T = all_ranking.T

# Add column names
all_rankings_T.columns = ['Total 5 star', 'Total 4 star', 'Total 3 star']
all_rankings_T.reset_index(inplace=True)

# Show data
all_rankings_T

In [ ]:
%matplotlib notebook

In [ ]:
# Create bar chart for numbers of 5 and 4 star prospects
%matplotlib inline 

# Create ordered list of position labels for x-axis
labels = ['Quarterback', 'Running Back', 'Wide Receiver', 'Offensive Line', 'Def Line', 'Linebacker', 'Cornerback']

# Copy all_rankings_T df
bar_chart_df = all_rankings_T.copy()

# Create mapping based on order of labels
bar_chart_df['mapped_position'] = pd.Categorical(bar_chart_df['mapped_position'], categories=labels)

# Sort df by ordered labels
bar_chart_df.sort_values(by='mapped_position', inplace=True)
#bar_chart_df

# Create plot
fig, ax = plt.subplots(figsize=(14, 5))

# Create list of labels for x-axis
x = np.arange(len(labels))

# Set width of bars
width = 0.4

# Create 5 and 4 star bars separately
rects1 = ax.bar(x - width / 2, bar_chart_df['Total 5 star'], width, label='5-star')
rects2 = ax.bar(x + width / 2, bar_chart_df['Total 4 star'], width, label='4-star')

# Format chart
ax.set_ylabel('Total Players', fontsize=24)
ax.set_ylim(0, 800)

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlabel('Positions', fontsize=24)

ax.tick_params(labelsize=16)

# Create legend
ax.legend(loc='best', bbox_to_anchor=(0.075, 0.25, 0.75, 0.75), fontsize=24)

# Remove lines at top and right of chart
ax.spines[['right', 'top']].set_visible(False)

# Add values above bars
ax.bar_label(rects1, padding=3, fontsize=26)
ax.bar_label(rects2, padding=3, fontsize=26)

fig.tight_layout()

# Save chart to png file
#plt.savefig('5&4 grouped bar.png')

# Show plot
plt.show()

# Calculate total number of recruits from each state

In [ ]:
# Groupby state -- how many 5/4/3 star recruits from each state?

# Groupby to get counts by state
state = final.groupby(['state','stars'])['stars'].count()
print(len(final))

# Turn into df
state_df = pd.DataFrame(state)

# Rename columns
state_df = state_df.rename(columns={'stars':'total players'})

# Reset index
state_df.reset_index('stars', inplace=True)

# Review df
#state_df.head(5)

# Pivot df so star ratings become columns
state_pivot = state_df.pivot(columns='stars')

# Flatten df
state_pivot.columns = state_pivot.columns.get_level_values(1)

# Add sum of each state
state_pivot['total recruits'] = state_pivot.sum(axis=1)

# Sort by total number of recruits
most_stars = state_pivot.sort_values(by='total recruits', ascending=False)

# Reset index
most_stars.reset_index(inplace=True)

# Calculate each state's % of total recruits
most_stars['% of total recruits'] = (most_stars['total recruits'] / sum(most_stars['total recruits'])).multiply(100).round(1)

# Calculate % of each state's recruits that are 5 star
most_stars['5 % of total'] = (most_stars[5] / most_stars['total recruits']).multiply(100).round(2)

# Show table
most_stars.head()

# Check at individual position level (rather than position group level) whether there are any trends showing that average income is significantly higher for higher star ratings.

In [ ]:
# Part A: Count total number of 5, 4, 3 star prospects for each individual position

# 5 star recruits ------------------------------------------------------

# Take subset of df for only 5-star propsects
x_stars_5 = final[final['stars'] == 5]
print("Total 5 star Recruits (all positions): ", len(x_stars_5))

# Groupby position and count
position_totals_5 = x_stars_5.groupby('position')['position'].count()


# 4 star recruits ------------------------------------------------------

# Take subset of df for only 4-star propsects
x_stars_4 = final[final['stars'] == 4]
print("Total 4 star Recruits (all positions): ", len(x_stars_4))

# Groupby position and count
position_totals_4 = x_stars_4.groupby('position')['position'].agg('count')


# 3 star recruits ------------------------------------------------------

# Take subset of df for only 3-star propsects
x_stars_3 = final[final['stars'] == 3]
print("Total 3 star Recruits (all positions): ", len(x_stars_3))

# Groupby position and count
position_totals_3 = x_stars_3.groupby('position')['position'].count()


# Combine 5, 4 and 3 star data-------------------------------------------------- 
all_star_ranking = pd.DataFrame([position_totals_5,
                                 position_totals_4,
                                 position_totals_3])

# Transpose to get star rankings in columns
all_star_rankings_T = all_star_ranking.T

# Add column names
all_star_rankings_T.columns = ['Total 5 star', 'Total 4 star', 'Total 3 star']

# Show results
all_star_rankings_T.head()

In [ ]:
# Spot check 5-star, PRO quarterbacks

# Take subset of all PRO-style QBs
only_qb = final[final['position'] == 'PRO']

# Get 5-star PRO qbs only
five_qb = only_qb[only_qb['stars'] == 5]

# Review list
five_qb.head()

In [ ]:
# Part B -- Calculate average income by star ranking

# 5 star----------------------------------------------------

# Get subset of 5-star recruits
x_stars_5 = final[final['stars'] == 5]

# Group by position, get mean Average Income
position_income_5_mean = (pd.DataFrame(x_stars_5.groupby(['position'])['Average Income'].mean().divide(1000))).round(2)

# Rename column
position_income_5_mean.columns = ['5 star mean inc']
#position_income_5_mean

# 4 star----------------------------------------------------

# Get subset of 4-star recruits
x_stars_4 = final[final['stars']==4]

# Group by position, get mean Average Income
position_income_4_mean = (pd.DataFrame(x_stars_4.groupby(['position'])['Average Income'].mean().divide(1000))).round(2)

# Rename column
position_income_4_mean.columns = ['4 star mean inc']
#position_income_4_mean

# 3 star----------------------------------------------------

# Get subset of 3-star recruits
x_stars_3 = final[final['stars'] == 3]

# Group by position, get mean Average Income
position_income_3_mean = (pd.DataFrame(x_stars_3.groupby(['position'])['Average Income'].mean().divide(1000))).round(2)

# Rename column
position_income_3_mean.columns = ['3 star mean inc']
#position_income_3_mean

# Combine 5,4 and 3 star rankings --------------------------

# Combine data for 5, 4 and 4 star rankings
position_avg_inc_all = pd.concat([position_income_5_mean,position_income_4_mean,position_income_3_mean], axis=1)

# Calculate % change between Average Income of 5 star v 4 star, and 4 star v 3 star
position_avg_inc_all['5 v 4'] = (position_avg_inc_all['5 star mean inc'] / position_avg_inc_all['4 star mean inc'] - 1).round(2)
position_avg_inc_all['4 v 3'] = (position_avg_inc_all['4 star mean inc'] / position_avg_inc_all['3 star mean inc'] - 1).round(2)

# Review data                          
position_avg_inc_all.head()

In [ ]:
# Part C Combine Results -- total number of 5,4,3 star prospects AND mean income 
# for each position, plus % changes btw 5 and 4-star avg inc and 4 and 3-star avg inc

# Combine df from Part A (# of recruits) with Part B (Avg Inc)
player_avg_inc = pd.concat([all_star_rankings_T,position_avg_inc_all], axis=1)

# Sort df by 4-star mean income
player_avg_inc.sort_values(by='4 star mean inc', ascending=False)

# Review first few items
player_avg_inc.head()

# Aggregate level analysis -- Check distribution of recruits across each star rating and income quartile

In [ ]:
# Aggregate level data -- all positions

# Group by quintiles and stars, then get counts
all_positions = (pd.DataFrame(final.groupby(['stars', 'Quartile Level'])['Quartile Level'].count()))

# Add % of each subgroup column
all_positions['Pct'] = (all_positions['Quartile Level'] / 
                        all_positions.groupby('stars')['Quartile Level'].transform('sum')).round(2)

# Show only 5 and 4 star recruit data
all_positions.tail(10)

# Calculate same breakdown of star rankings and income quartiles at position group level

Create function to calculate number of recruits at each star level and income bracket.    Also calculate what % of recruits are from a particular income quartile for a given star level.

In [ ]:
def star_quartile_by_position(indf,indiv_positions):

    # Filter by position
    position_df = indf[indf['position'].isin(indiv_positions)]
    print("Total number in position group:", len(position_df))

    # Show value counts by star ranking for reference
    print(position_df['stars'].value_counts())

    # Groupby quartiles and stars, get counts of number of recruits in each combination of star & quartile
    position_quartiles_stars = (pd.DataFrame(position_df.groupby(['stars','Quartile Level'])['Quartile Level'].count()))

    # Add column to show % of recruits from each income quartile within each star level
    position_quartiles_stars['Pct'] = (position_quartiles_stars['Quartile Level'] 
                                       / position_quartiles_stars.groupby('stars')['Quartile Level'].transform('sum')).round(2)

    return position_quartiles_stars

# Offensive Position Groups

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for QB
qb_stars_quartiles = star_quartile_by_position(final, ['QB', 'DUAL', 'PRO'])

# Show 5 and 4-star data only
qb_stars_quartiles[-10:]

# Create function to run t-test for a given position group

In [ ]:
def t_test_calc(indf,indiv_positions):
    
    # Calc mean Average Income for all players at chosen position
    avg_inc_for_position = indf['Average Income'].mean()
    print("Average Income for Position: ", avg_inc_for_position)

    # Make copy of final df
    final_q_copy = final.copy()

    # Exclude all players at chosen position from broader df
    exclude_position_df = final_q_copy[~final_q_copy['position'].isin(indiv_positions)]

    # Calc mean of avg inc for all remaining prospects
    exclude_position_avg_inc = exclude_position_df['Average Income'].mean()
    print("All Mean: ", exclude_position_avg_inc)

    # Use t-test to see if means are equal (Ho) 
    
    # Calculate test stat and p-value
    stat,p = ttest_ind(indf['Average Income'], exclude_position_df['Average Income'])
    print("T-stat", stat, "p-value", p)

    # Interpretation
    alpha = 0.05

    if p > alpha:
        print("Same distributions (fail to reject Ho)")
    else:
        print("Different dist (reject Ho)")


In [ ]:
# Create subset of only QBs
all_qb_df = final[final['position'].isin(['QB', 'DUAL', 'PRO'])]

# Review df
all_qb_df.head()

# Get length of df
print(len(all_qb_df))

In [ ]:
# Run t-test for Quarterbacks
qb_ttest = t_test_calc(all_qb_df, ['QB', 'DUAL', 'PRO'])
qb_ttest

In [ ]:
# Also test using Mann-Whitney U test -- More suitable for non-normally distributed data

def mann_whitney_calc(indf,indiv_positions):
    
    # Calculate mean Average Income for all players at chosen position
    avg_inc_for_position = indf['Average Income'].mean()
    print("Average Income for Position: ", avg_inc_for_position)

    # Make copy of final_q df
    final_q_copy = final.copy()

    # Exclude all players at chosen position from broader df
    exclude_position_df = final_q_copy[~final_q_copy['position'].isin(indiv_positions)]

    # Calc mean of Average Income for all remaining prospects
    exclude_position_avg_inc = exclude_position_df['Average Income'].mean()
    print("All Mean: ", exclude_position_avg_inc)

    # Use t-test to see if means are equal (Ho) 
    
    # Calculate test stat and p value
    stat, p = mannwhitneyu(indf['Average Income'], exclude_position_df['Average Income'])
    print("Mann-Whitney Stat", stat, "p-value", p)

    # Interpretation
    alpha = 0.05

    if p > alpha:
        print("Same distributions (fail to reject Ho)")
    else:
        print("Different dist (reject Ho)")

In [ ]:
# Run t-test for Quarterbacks
qb_mann = mann_whitney_calc(all_qb_df, ['QB', 'DUAL', 'PRO'])
qb_mann

# Confirm graphically whether the distribution of income for QBs looks different compared to all other recruits

In [ ]:

def inc_dist_comparison(indf,indiv_positions):

    # Get subset of players at chosen position
    position_subset_df = indf[indf['position'].isin(indiv_positions)]
    
    # Make copy of final_q df
    final_q_copy = final.copy()

    # Exclude all players at chosen position from broader df
    all_other_positions_df = final_q_copy[~final_q_copy['position'].isin(indiv_positions)]
    
    # Create chart
    
    # Create figure
    fig,ax = plt.subplots(figsize=(15, 5))

    # Income Distribution of all players EXCLUDING chosen position
    all_other_position_inc_dist = all_other_positions_df['Average Income'].plot(kind='kde', ax=ax, linewidth=5, color='blue')

    # Set up second axis
    ax2 = ax.twinx()

    # Income Distibution for chosen position
    position_inc_dist = position_subset_df['Average Income'].plot(kind='kde',ax=ax2,ls='--',color='red',linewidth=3)

    # Format chart and show
    plt.legend(loc='upper right')
    fig.tight_layout()
    plt.show()

In [ ]:
# Compare distribution of income for Quarterbacks vs all other players
qb_inc_chart = inc_dist_comparison(final, ['QB', 'PRO', 'DUAL'])

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Running Backs
rb_stars_quartiles = star_quartile_by_position(final, ['RB', 'FB', 'APB'])

# Show 5 and 4 star data
rb_stars_quartiles[-10:]

In [ ]:
# Create subset of only RBs
all_rb_df = final[final['position'].isin(['RB', 'FB', 'APB'])]

# Review df
all_rb_df.head()

# Get length of df
print(len(all_rb_df))

In [ ]:
# Calculate t-test for Running Backs
rb_ttest = t_test_calc(all_rb_df,['RB', 'FB', 'APB'])
rb_ttest

In [ ]:
# Calculate Mann-Whitney U test for Running Backs
rb_mann = mann_whitney_calc(all_rb_df,['RB', 'FB', 'APB'])
rb_mann

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Wide Receivers
wr_stars_quartiles = star_quartile_by_position(final, ['WR', 'TE'])
 
# Show results for 5 and 4 star recruits only
wr_stars_quartiles[-10:]

In [ ]:
# Create subset of only Wide Receivers
all_wr_df = final[final['position'].isin(['WR', 'TE'])]

# Review df
all_wr_df.head()

# Get length of df
print(len(all_wr_df))

In [ ]:
# Calculate t-test for Wide Receivers
wr_ttest = t_test_calc(all_wr_df,['WR', 'TE'])
wr_ttest

In [ ]:
# Calculate Mann-Whitney U test for Wide Receivers
wr_mann = mann_whitney_calc(all_wr_df,['WR', 'TE'])
wr_mann

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Offensive Linemen
ol_stars_quartiles = star_quartile_by_position(final, ['IOL', 'OC', 'OG', 'OT'])

# Show results for 5 and 4 star recruits only
ol_stars_quartiles[-10:]

In [ ]:
# Create subset of only Offensive Linemen
all_ol_df = final[final['position'].isin(['IOL', 'OC', 'OG', 'OT'])]

# Review df
all_ol_df.head()

# Get length of df
print(len(all_ol_df))

In [ ]:
# Calculate t-test for Offensive Linemen
ol_ttest = t_test_calc(all_ol_df,['IOL', 'OC', 'OG', 'OT'])
ol_ttest

In [ ]:
# Calculate Mann-Whitney U test for Offensive Linemen
ol_mann = mann_whitney_calc(all_ol_df,['IOL', 'OC', 'OG', 'OT'])
ol_mann

In [ ]:
# Compare distribution of income for Offensive Linemen vs all other players
ol_inc_chart = inc_dist_comparison(final,['IOL', 'OC', 'OG', 'OT'])

In [ ]:
# Combine all offensive position groups - QB, RB, WR and OL

# Combine results from QB, RB, WR and OL analysis
combined_positions_off = pd.concat([qb_stars_quartiles, rb_stars_quartiles, wr_stars_quartiles, ol_stars_quartiles], axis=1)

# Add column names
combined_positions_off.columns = ['QB Total', 'QB %', 'RB Total', 'RB %', 'WR Total', 'WR %', 'OL Total', 'OL %']

# Show only 5 and 4 star data
combined_positions_off.tail(10)

# Defensive position groups

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Defensive Linemen
dl_stars_quartiles = star_quartile_by_position(final, ['EDGE', 'DL', 'WDE', 'SDE', 'DT'])
#dl_stars_quartiles

In [ ]:
# Create subset of only Defensive Linemen
all_dl_df = final[final['position'].isin(['EDGE', 'DL', 'WDE', 'SDE', 'DT'])]

# Review df
all_dl_df.head()

# Get length of df
print(len(all_dl_df))

In [ ]:
# Calculate t-test for Defensive Linemen
dl_ttest = t_test_calc(all_dl_df,['EDGE', 'DL', 'WDE', 'SDE', 'DT'])
dl_ttest

In [ ]:
# Calculate Mann-Whitney U tset for Defensive Linemen
dl_mann = mann_whitney_calc(all_dl_df, ['EDGE', 'DL', 'WDE', 'SDE', 'DT'])
dl_mann

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Linebackers
lb_stars_quartiles = star_quartile_by_position(final, ['LB', 'ILB', 'OLB'])
#lb_stars_quartiles

In [ ]:
# Create subset of only Linebackers
all_lb_df = final[final['position'].isin(['LB', 'ILB', 'OLB'])]

# Review df
all_lb_df.head()

# Get length of df
print(len(all_lb_df))

In [ ]:
# Calculate t-test for Linebackers
lb_ttest = t_test_calc(all_lb_df,['LB', 'ILB', 'OLB'])
lb_ttest

In [ ]:
# Calculate Mann-Whitney U test for Linebackers
lb_mann = mann_whitney_calc(all_lb_df, ['LB', 'ILB', 'OLB'])
lb_mann

In [ ]:
# Compare distribution of income for Linebackers vs all other players
lb_inc_chart = inc_dist_comparison(final, ['LB', 'ILB', 'OLB'])

In [ ]:
# Get value counts by star ranking, totals in each income quartile and % for Cornerbacks
cb_stars_quartiles = star_quartile_by_position(final, ['S', 'CB'])

# Show results for 5 and 4-star recruits only
#cb_stars_quartiles[-10:]

In [ ]:
# Create subset of only Cornerbacks
all_cb_df = final[final['position'].isin(['S', 'CB'])]

# Review df
all_cb_df.head()

# Get length of df
print(len(all_cb_df))

In [ ]:
# Calculate t-test for Cornerbacks
cb_ttest = t_test_calc(all_cb_df, ['S', 'CB'])
cb_ttest

In [ ]:
# Calculate Mann-Whitney U test for Cornerbacks
cb_mann = mann_whitney_calc(all_cb_df, ['S', 'CB'])
cb_mann

In [ ]:
# Compare distribution of income for Cornerbacks vs all other players
cb_inc_chart = inc_dist_comparison(final, ['S', 'CB'])

In [ ]:
# Combine all defensive position groups

# Combine results from DL, LB and CB analysis
combined_positions_def = pd.concat([dl_stars_quartiles, lb_stars_quartiles, cb_stars_quartiles], axis=1)

# Add column names
combined_positions_def.columns = ['DL Total', 'DL %', 'LB Total', 'LB %', 'CB Total', 'CB %']

# Show only 5 and 4 star data
combined_positions_def.tail(10)

# Analyze income distribution by individual colleges or conferences

# Create function to analyze any single college's recruits

In [ ]:
def indiv_college_analyzer(indf,college):

    # Filter by school
    school_x = indf[indf['committedTo'] == college]
    
    # Show how many recruits chosen college has in total
    print("Total of recruits for chosen school: ", len(school_x))
    #school_x.head()

    # Show value counts by star ranking for reference
    print(school_x['stars'].value_counts())

    # Groupby quintiles and stars and get counts
    school_x_quintiles_stars = (pd.DataFrame(school_x.groupby(['stars', 'Quartile Level'])['Quartile Level'].count()))

    # Add % of each subgroup
    school_x_quintiles_stars['Pct'] = (school_x_quintiles_stars['Quartile Level'] 
                                       / school_x_quintiles_stars.groupby('stars')['Quartile Level'].transform('sum')).round(2)

    return school_x_quintiles_stars

In [ ]:
# Calculate star / income quartile breakdown for an individual school
georgia = indiv_college_analyzer(final, 'Georgia')

# Show only 5 and 4 star totals
georgia[-10:]

# Calculate more detailed breakdown at position rather then position group level

In [ ]:
# Analyze a given school's recruits at position (not position group) level

school = 'Georgia'

# Filter by school
school_x = final[final['committedTo'] == school]
print("len of school", len(school_x))
#school_x.head()

# Show value counts by star ranking
print(school_x['stars'].value_counts())

# Groupby quintiles and stars, get mean income
school_x_quintiles_stars = (pd.DataFrame(school_x.groupby(['stars', 'Quartile Level', 'position'])['Quartile Level'].count()))

# Add % of each subgroup
school_x_quintiles_stars['Pct'] = (school_x_quintiles_stars['Quartile Level'] 
                                   / school_x_quintiles_stars.groupby('stars')['Quartile Level'].transform('sum')).round(2)

#school_x_quintiles_stars[-50:]

In [ ]:
# Calculate star / income quartile breakdown for an individual school
ucla = indiv_college_analyzer(final, 'UCLA')

# Show only 5 and 4 star totals
ucla[-9:]

# Conference level analysis - check aggregate distribution of recruits across an entire athletic conference

In [ ]:
# Create dictionary of SEC schools
sec_schools = {'Alabama':'SEC',
              'Georgia':'SEC',
              'LSU':'SEC',
              'Florida':'SEC',
              'Tennessee':'SEC',
              'South Carolina':'SEC',
              'Vanderbilt':'SEC',
              'Ole Miss':'SEC',
              'Arkansas':'SEC',
              'Auburn':'SEC',
              'Mississippi State':'SEC',
              'Kentucky':'SEC',
              'Missouri':'SEC',
              'Texas A&M':'SEC'}

# Create dictionary of Pac12 schools
pac12_schools = {'Arizona':'Pac12',
              'Arizona State':'Pac12',
              'Colorado':'Pac12',
              'Oregon':'Pac12',
              'Oregon State':'Pac12',
              'UCLA':'Pac12',
              'USC':'Pac12',
              'Stanford':'Pac12',
              'Utah':'Pac12',
              'Washington':'Pac12',
              'Washington State':'Pac12',
              }


# Create copy of df
conf_df = final.copy()

# Create new conference column and map school names
conf_df['conference'] = conf_df['committedTo'].map(sec_schools)

# Select conference to analyze
conf = 'SEC'

# Filter by teams in chosen conference
conf_x = conf_df[conf_df['conference'] == conf]

# Show value counts by star ranking
print(conf_x['stars'].value_counts())

# Groupby quintiles and stars, get counts
conf_x_quintiles_stars = (pd.DataFrame(conf_x.groupby(['stars', 'Quartile Level'])['Quartile Level'].count()))

# Add % of each subgroup
conf_x_quintiles_stars['Pct'] = (conf_x_quintiles_stars['Quartile Level'] 
                                 / conf_x_quintiles_stars.groupby('stars')['Quartile Level'].transform('sum')).round(2)

# Add % of each subgroup
conf_x_quintiles_stars['Pct'] = (conf_x_quintiles_stars['Quartile Level'] 
                                 / conf_x_quintiles_stars.groupby('stars')['Quartile Level'].transform('sum')).round(2)

# Show results
conf_x_quintiles_stars[-10:]


# Analyze trends in average income profile of select schools over time

In [ ]:
# Get Average Income of recruiting class for chosen colleges each year over time and make line plot

# Groupby school and year, get median Average Income
yearly = final.groupby(['committedTo','year'])['Average Income'].median()

# Make into df
yearly_df = pd.DataFrame(yearly)
yearly_df.reset_index(inplace=True)

# Pivot so that schools are in columns
pivoted = yearly_df.pivot(index='year',columns='committedTo')

# Get rid of multi-indexing
pivoted.columns = pivoted.columns.get_level_values(1)

# Select lists of schools to subset
#schools = ['Georgia','Alabama','LSU']
#schools = ['Stanford','Vanderbilt','Duke','Northwestern','Rice']
schools = ['Georgia','UCLA','Vanderbilt']
#schools = ['Ohio State','Michigan']
#schools = ['UCLA','USC']
#schools = ['Oklahoma','Texas']

# Get schools for line plots based on list above
subset = (pivoted[schools]).divide(1000).round(1)

# Set col names to match school names
subset.columns = schools
#subset.head()

# Create line plot for each school
# Create figure
fig, ax = plt.subplots(figsize=(14, 7))

# Create line plots
lines = sns.lineplot(data=subset, linewidth=5)

# Create and format legend
leg = ax.legend()
for line in leg.get_lines():
    line.set_linewidth(6.0)
plt.setp(lines.get_legend().get_texts(), fontsize='30')

In [ ]:
# Calculate highest Average Income by school

# Get data for most recent recruiting class available
most_recent = final[final['year'] == BEGIN_YEAR]

# Groupby team, get mean Average Income
avg_inc_sort = (most_recent.groupby('committedTo')['Average Income'].mean()).divide(1000).round(1)

# Create df
avg_inc_df = pd.DataFrame(avg_inc_sort.sort_values(ascending=False))
avg_inc_df.reset_index(inplace=True)

# Show top 10
avg_inc_df.head(10)

# Getting high star recruits average 3.5 stars and above for each team 

In [ ]:
number_recruits_team = final.groupby(['committedTo']).size().reset_index()

number_recruits_team.rename(columns = {0:'count'}, inplace = True)

sum_stars_team = final.groupby(['committedTo'])['stars'].sum().reset_index()



high_star_players = sum_stars_team.merge(number_recruits_team, left_on='committedTo', right_on='committedTo')

high_star_players = high_star_players[high_star_players['stars'] / high_star_players['count'] >= 3.5 ]

high_star_players.head()


# Plotting all players on the map and colors represents the income level

In [ ]:
gdf = geopandas.GeoDataFrame(final, geometry=geopandas.points_from_xy(final['hometownInfo.longitude'], final['hometownInfo.latitude']))
gdf.head()

In [ ]:

count_committedTo_income = pd.DataFrame({'count' : final.groupby(["Quartile Level"]).size()}).reset_index()
count_committedTo_income['percent_quintile'] = count_committedTo_income['count'] / count_committedTo_income['count'].sum()

print(count_committedTo_income['count'].sum())

count_committedTo_income.head()



In [ ]:

def geo_plot(gdf):
  px.set_mapbox_access_token("pk.eyJ1IjoidGF3ZmlxeiIsImEiOiJjbGRldGVxOTMwZ3k2M3JxcG5qOGNlb3p6In0.tE9WHVe2oHaysmx7dMov7w")

  fig = px.scatter_mapbox(gdf,
                          lat=gdf.geometry.y,
                          lon=gdf.geometry.x,
                          hover_name="name",
                          hover_data=["rating", "year", "school"],
                          color="Quartile Level",
                          zoom=1)

  fig.update_layout(legend=dict(font=dict(size= 20), bordercolor='rgb(100,100,100)',
                                  borderwidth=2,
                                  itemclick= 'toggleothers',# when you are clicking an item in legend all that are not in the same group are hidden
                                  x=0.750,
                                  y=1))

  fig.update_layout(mapbox_style="open-street-map")

  fig.show()

geo_plot(gdf[gdf['stars'] >= 4])

# Plotting all players on the map and colors represents the income level

In [ ]:
geo_plot(gdf[gdf['stars'] >= 1])

# plotting a map plot filterd by a specific team 

In [ ]:
#plotting here for a team that has players with an average of 3.5 stars and above

geo_plot(gdf[gdf['committedTo'] == 'UCLA'])


In [ ]:
#plotting for any team without filter on stars only filtered by the difference in income levels

geo_plot(gdf[gdf['committedTo'] == "Georgia"])

# plotting a bar chart to see how the same team is performing in terms of recruiting based on the stars 

In [ ]:

def bar_on_team(team="UCLA"):
    
  ex = final[final['committedTo'].astype(str).str.contains(team)][['stars', 'name', 'school']]

  gp_chart = alt.Chart(ex).mark_bar().encode(
    alt.X('name'),
    alt.Y('stars', axis=alt.Axis(grid=False)), 
    )
    
  gp_chart.display()

bar_on_team()

In [ ]:
bar_on_team("Georgia")